In [1]:
#import dependencies
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io
import os
import re
from time import sleep

In [2]:
# Time series data start and end date
start = datetime.datetime(2015,1,1)
end = datetime.datetime(2022,3,16)

In [3]:
file_to_load = os.path.join("Resources", "ESG_Companies.csv")
ESG_df = pd.read_csv(file_to_load)
ESG_df

,NAME,SYMBOL,YFINANCE_ESG,YFINANCE_E,YFINANCE_S,YFINANCE_G,MSCI,JUST_100
0,Intel Corp,INTC,17.0,4.4,5.6,6.9,AA,2
1,Salesforce.Com Inc,CRM,13.0,2.3,6.5,4.4,AA,4
2,Verizon Communications Inc,VZ,17.0,1.7,9.3,5.7,AA,9
3,Cisco Systems Inc,CSCO,12.0,0.0,5.6,6.2,AA,10
4,PepsiCo Inc.,PEP,16.0,4.9,6.6,4.6,AA,12
5,Microsoft Corp,MSFT,13.0,0.5,8.1,4.7,AAA,3
6,NVIDIA Corp,NVDA,13.0,2.5,4.1,6.1,AAA,8
7,Alphabet Inc.,GOOG,NaN,NaN,NaN,NaN,NaN,1
8,Excelon Inc.,EXC,24.0,9.1,9.0,6.3,A,11
9,PayPal Inc,PYPL,16.0,0.7,9.9,5.8,A,6


In [4]:
Symbols = ESG_df['SYMBOL'].tolist()
Symbols

['INTC', 'CRM', 'VZ', 'CSCO', 'PEP', 'MSFT', 'NVDA', 'GOOG', 'EXC', 'PYPL']

In [5]:
# create empty dataframe
stocks_df = pd.DataFrame()
# iterate over each symbol
for i in Symbols:  
    
    # print the symbol which is being downloaded
    print( str(Symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stocks_df = stocks_df.append(stock,sort=False)
            print(f'{i}: data appended')
    except Exception:
        None

0 : INTC,INTC: data appended
1 : CRM,CRM: data appended
2 : VZ,VZ: data appended
3 : CSCO,CSCO: data appended
4 : PEP,PEP: data appended
5 : MSFT,MSFT: data appended
6 : NVDA,NVDA: data appended
7 : GOOG,GOOG: data appended
8 : EXC,EXC: data appended
9 : PYPL,PYPL: data appended


In [6]:
stocks_df

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2015-01-02,36.669998,37.160000,36.110001,36.360001,29.852991,23605600,INTC
2015-01-05,36.200001,36.450001,35.939999,35.950001,29.516354,32785000,INTC
2015-01-06,36.080002,36.230000,35.259998,35.279999,28.966257,30847600,INTC
2015-01-07,35.639999,36.070000,35.270000,36.020000,29.573839,27725200,INTC
2015-01-08,36.369999,37.000000,36.259998,36.689999,30.123932,31765400,INTC
...,...,...,...,...,...,...,...
2022-03-09,98.120003,100.730003,97.290001,100.220001,100.220001,20811300,PYPL
2022-03-10,99.050003,99.320000,96.400002,98.550003,98.550003,14703200,PYPL
2022-03-11,99.750000,99.820000,95.519997,96.570000,96.570000,15117200,PYPL


In [7]:
from bs4 import BeautifulSoup
import requests
import ipywidgets as widgets
from ipywidgets import interact

In [8]:
# To fill zeros where the values are not available
def profile_fill_zero(profile):
    end_profile = []

    i = 0
    while i < (len(profile)):
    #     print(temp_net_impact_profile[i][0])
    #     print(temp_net_impact_profile[i+1][0])
        if profile[i][0] == '-' and profile[i+1][0] == '+':
            end_profile.append(profile[i])
            end_profile.append(profile[i+1])
    #         print('-+')
            i += 2
        elif profile[i][0] == '-' and profile[i+1][0] == '-':
            end_profile.append(profile[i])
            end_profile.append(0.0)
    #         print('--')
            i += 1
        elif profile[i][0] == '+':
            end_profile.append(0.0)
            end_profile.append(profile[i])
    #         print('+')
            i +=1
    print(len(end_profile))
    return end_profile

In [9]:
def net_impact_profile(soup):
    temp = []
    for value in soup.findAll('div', {'class': "score-value position-absolute"}):
        temp.append(value.text)

    print(len(temp))
    net_impact_profile = profile_fill_zero(temp)
    return net_impact_profile

In [10]:
def final_profile_dict(net_impact_profile, value_sets):
    final_profile = {'Society_N': float(net_impact_profile[0]),
                                'Society_P': float(net_impact_profile[1]),
                                'Jobs_N': float(net_impact_profile[2]),
                                'Jobs_P': float(net_impact_profile[3]),
                                'Taxes_N':float(net_impact_profile[4]),
                                'Taxes_P':float(net_impact_profile[5]),
                                'Societal_Infrastructure_N':float(net_impact_profile[6]),
                                'Societal_Infrastructure_P':float(net_impact_profile[7]),
                                'Societal_Stability_N':float(net_impact_profile[8]),
                                'Societal_Stability_P':float(net_impact_profile[9]),
                                'EQUALITY_HUMAN RIGHTS_N':float(net_impact_profile[10]),
                                'EQUALITY_HUMAN RIGHTS_P': float(net_impact_profile[11]),
                                'Knowledge_N': float(net_impact_profile[12]),
                                'Knowledge_P': float(net_impact_profile[13]),
                                'Knowledge_Infrastructure_N':float(net_impact_profile[14]),
                                'Knowledge_Infrastructure_P':float(net_impact_profile[15]),
                                'Creating_Knowledge_N': float(net_impact_profile[16]),
                                'Creating_Knowledge_P': float(net_impact_profile[17]),
                                'Distributing_Knowledge_N': float(net_impact_profile[18]),
                                'Distributing_Knowledge_P': float(net_impact_profile[19]),
                                'Scarce_Human_Capital_N': float(net_impact_profile[20]),
                                'Scarce_Human_Capital_P': float(net_impact_profile[21]),
                                'Health_N': float(net_impact_profile[22]),
                                'Health_P': float(net_impact_profile[23]),
                                'Physical Diseases_N': float(net_impact_profile[24]),
                                'Physical Diseases_P': float(net_impact_profile[25]),
                                'Mental Diseases_N': float(net_impact_profile[26]),
                                'Mental Diseases_P': float(net_impact_profile[27]),
                                'Nutrition_N': float(net_impact_profile[28]),
                                'Nutrition_P': float(net_impact_profile[29]),
                                'Relationships_N': float(net_impact_profile[30]),
                                'Relationships_P': float(net_impact_profile[31]),
                                'Meaning_Joy_N': float(net_impact_profile[32]),
                                'Meaning_Joy_P': float(net_impact_profile[33]),
                                'Environment_N': float(net_impact_profile[34]),
                                'Environment_P': float(net_impact_profile[35]),
                                'GHG_Emissions_N': float(net_impact_profile[36]),
                                'GHG_Emissions_P': float(net_impact_profile[37]),
                                'Non-GHG_Emissions_N': float(net_impact_profile[38]),
                                'Non-GHG_Emissions_P': float(net_impact_profile[39]),
                                'Scarce_Natural_resources_N': float(net_impact_profile[40]),
                                'Scarce_Natural_resources_P': float(net_impact_profile[41]),
                                'Biodiversity_N': float(net_impact_profile[42]),
                                'Biodiversity_P': float(net_impact_profile[43]),
                                'Waste_N': float(net_impact_profile[44]),
                                'Waste_P': float(net_impact_profile[45]),
                                'Millennials': float(value_sets[10][:-1])/100,
                                'Most Wanted Workforce': float(value_sets[11][:-1])/100,
                                'Society First': float(value_sets[11][:-1])/100,
                                'Knowledge First': float(value_sets[12][:-1])/100,
                                'Health First': float(value_sets[13][:-1])/100
                               }

                                                 

    return final_profile

In [11]:
def value_sets(soup):
    temp_value_sets = []
    for value in soup.find_all("dd"):
        temp_value_sets.append(value.text)
    
    return temp_value_sets

In [12]:
ESG_data = {}
for symbol in Symbols:
    try:
        web_data = requests.get(f'https://www.marketbeat.com/stocks/NASDAQ/{symbol}/sustainability/').text

        print(f'Received {symbol} working on it ...')
        soup = BeautifulSoup(web_data, 'html.parser')

        # Net impact profile
        temp_impact_profile = []
        for value in soup.findAll('div', {'class': "score-value position-absolute"}):
            temp_impact_profile.append(value.text)
        #print(len(temp_impact_profile))
        net_impact_profile = profile_fill_zero(temp_impact_profile)

        # value sets
        value_sets = []
        for value in soup.find_all("dd"):
            value_sets.append(value.text)

        # Including both impact profile and value sets
        final_profile = final_profile_dict(net_impact_profile, value_sets)
        ESG_data.update({symbol: final_profile})
        
    except:
        print(f'{symbol} did not have sustainability data skipping ...')

    sleep(10)


Received INTC working on it ...
46
Received CRM working on it ...
0
CRM did not have sustainability data skipping ...
Received VZ working on it ...
0
VZ did not have sustainability data skipping ...
Received CSCO working on it ...
46
Received PEP working on it ...
46
Received MSFT working on it ...
46
Received NVDA working on it ...
46
Received GOOG working on it ...
46
Received EXC working on it ...
46
Received PYPL working on it ...
46


In [14]:
ESG_data

{'INTC': {'Society_N': -0.06,
  'Society_P': 2.25,
  'Jobs_N': 0.0,
  'Jobs_P': 0.68,
  'Taxes_N': 0.0,
  'Taxes_P': 1.31,
  'Societal_Infrastructure_N': 0.0,
  'Societal_Infrastructure_P': 0.12,
  'Societal_Stability_N': -0.05,
  'Societal_Stability_P': 0.12,
  'EQUALITY_HUMAN RIGHTS_N': -0.01,
  'EQUALITY_HUMAN RIGHTS_P': 0.02,
  'Knowledge_N': -1.28,
  'Knowledge_P': 1.54,
  'Knowledge_Infrastructure_N': 0.0,
  'Knowledge_Infrastructure_P': 0.75,
  'Creating_Knowledge_N': 0.0,
  'Creating_Knowledge_P': 0.5,
  'Distributing_Knowledge_N': -0.0,
  'Distributing_Knowledge_P': 0.3,
  'Scarce_Human_Capital_N': -1.28,
  'Scarce_Human_Capital_P': 0.0,
  'Health_N': -0.2,
  'Health_P': 0.5,
  'Physical Diseases_N': -0.12,
  'Physical Diseases_P': 0.16,
  'Mental Diseases_N': -0.04,
  'Mental Diseases_P': 0.03,
  'Nutrition_N': 0.0,
  'Nutrition_P': 0.01,
  'Relationships_N': -0.03,
  'Relationships_P': 0.19,
  'Meaning_Joy_N': -0.01,
  'Meaning_Joy_P': 0.1,
  'Environment_N': -1.23,
  'Envir

In [15]:
ESG_Marketbeat_df = pd.DataFrame.from_dict(ESG_data)
ESG_Marketbeat_df

,INTC,CSCO,PEP,MSFT,NVDA,GOOG,EXC,PYPL
Society_N,-0.060,-0.100,-0.040,-0.280,-0.070,-0.060,-0.090,-0.020
Society_P,2.250,3.050,2.070,3.490,2.840,2.770,4.560,3.310
Jobs_N,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Jobs_P,0.680,0.840,0.950,1.060,0.910,0.850,0.880,0.540
Taxes_N,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Taxes_P,1.310,1.760,0.840,2.090,1.680,1.710,1.170,1.260
Societal_Infrastructure_N,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Societal_Infrastructure_P,0.120,0.160,0.280,0.180,0.130,0.160,2.500,1.390
Societal_Stability_N,-0.050,-0.100,-0.010,-0.270,-0.060,-0.040,-0.080,-0.010
Societal_Stability_P,0.120,0.280,0.000,0.130,0.100,0.040,0.010,0.050


In [16]:
ESG_Marketbeat_df = ESG_Marketbeat_df.transpose()
ESG_Marketbeat_df

,Society_N,Society_P,Jobs_N,Jobs_P,Taxes_N,Taxes_P,Societal_Infrastructure_N,Societal_Infrastructure_P,Societal_Stability_N,Societal_Stability_P,...,Scarce_Natural_resources_P,Biodiversity_N,Biodiversity_P,Waste_N,Waste_P,Millennials,Most Wanted Workforce,Society First,Knowledge First,Health First
INTC,-0.06,2.25,0.0,0.68,0.0,1.31,0.0,0.12,-0.05,0.12,...,0.0,-0.03,0.0,-0.30,0.00,0.528,0.327,0.327,0.291,0.775
CSCO,-0.10,3.05,0.0,0.84,0.0,1.76,0.0,0.16,-0.10,0.28,...,0.0,-0.02,0.0,-0.24,0.00,0.576,0.458,0.458,0.441,0.802
PEP,-0.04,2.07,0.0,0.95,0.0,0.84,0.0,0.28,-0.01,0.00,...,0.0,-0.07,0.0,-0.35,0.01,0.089,-0.721,-0.721,-0.297,-0.337
MSFT,-0.28,3.49,0.0,1.06,0.0,2.09,0.0,0.18,-0.27,0.13,...,0.0,-0.02,0.0,-0.16,0.00,0.394,0.183,0.183,0.201,0.690
NVDA,-0.07,2.84,0.0,0.91,0.0,1.68,0.0,0.13,-0.06,0.10,...,0.0,-0.02,0.0,-0.42,0.00,0.438,0.234,0.234,0.202,0.694
GOOG,-0.06,2.77,0.0,0.85,0.0,1.71,0.0,0.16,-0.04,0.04,...,0.0,-0.01,0.0,-0.15,0.00,0.564,0.430,0.430,0.370,0.823
EXC,-0.09,4.56,0.0,0.88,0.0,1.17,0.0,2.50,-0.08,0.01,...,0.0,-0.31,0.0,-1.32,0.11,0.563,0.280,0.280,0.470,0.375
PYPL,-0.02,3.31,0.0,0.54,0.0,1.26,0.0,1.39,-0.01,0.05,...,0.0,-0.01,0.0,-0.07,0.00,0.375,0.220,0.220,0.455,0.122
